In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from siglib.utils import load_dataset

cutoff = 100
notation = 'pyrimidine'
hg38_path = '/home/clint/buffalo/hg38/hg38.fa'
cosmic_mutants = '/home/clint/buffalo/cosmic/v76/CosmicMutantExport.tsv'
C10 = load_dataset('aflatoxin_10wk_average').values.flatten().tolist()

Load Human Genome into Memory for Fast Random Access
===

In [3]:
from siglib.utils import fasta_to_dict

hg38_path = '/home/clint/buffalo/hg38/hg38.fa'
hg38 = fasta_to_dict(hg38_path)

Parse all 301,076 COSMIC Samples
===

In [4]:
from siglib.cosmic import parse_to_samples

samples = parse_to_samples(cosmic_mutants)

Merge Samples from Same Tumours (digital replicates?)
===

1. Preserves clinical annotations (fills in data when missing)
2. Removes redundant mutations *per* sample as identified  by their COSMIC ID
3. Removes samples with less than `cutoff` mutations

In [5]:
from siglib.cosmic import merge_samples

merged_samples = merge_samples(samples, cutoff=cutoff)
print('{} Samples after Merging and Filtering'.format(len(merged_samples)))

6875 Samples after Merging and Filtering


Generate Trinucleotide Spectra for each Tumour Sample
===

1. Exome is used as trinucleotide normal
2. Mutations must be confirmed substitutions against GRCh v38
3. Mutations must come from the 23 chromosomes
4. Similarity to C10 is computed as `1 - cosine_distance`

In [6]:
from itertools import product
from siglib.jellywrap import jellywrap
from siglib.utils import spectrum_dict
from siglib.utils import dna_bases, substitution_label

from scipy.spatial.distance import cosine

merfile = '/home/clint/essigmann_analysis/mouse/data/9606_exons.jf'

j = jellywrap()
normal = dict(zip(*j.read_mer_file(merfile, notation=notation)))

for i, sample in enumerate(merged_samples.values()):
    if i % 1000 == 0: print('Completed sample: {}'.format(i));
    
    sample.spectrum = spectrum_dict(notation=notation)
    sample.total_mutations = 0
    sample.similarity = 0
    
    for mutation in sample.mutations:
        if (
                mutation.chrom != '' and
                mutation.chrom not in ['', 'chr25'] and
                mutation.GRCh == '38' and
                'Substitution' in mutation.description and
                'Confirmed' in mutation.somatic_status
           ):
            
            mutation.set_context(hg38, notation=notation)
            
        if (
                hasattr(mutation, 'label') and
                mutation.label != ''
            ):
        
            sample.spectrum[mutation.label] += 1
            sample.total_mutations += 1

    if sample.total_mutations == 0:
        continue
    
    # Normalize spectrum if num_mutations > 0
    for key, base in list(product(normal, dna_bases)):
        if base == key[1]:
            continue
        label = substitution_label.from_parts(''.join([key[1], '>', base]), key)

        sample.spectrum[label] = sample.spectrum[label] / normal[key]
    
    total = sum(sample.spectrum.values())

    # Rescale
    for key, value in sample.spectrum.items():
        sample.spectrum[key] = value / total
        
    # Compute distance from C10
    sample.similarity = 1 - cosine(C10, list(sample.spectrum.values()))

Completed sample: 0
Completed sample: 1000
Completed sample: 2000
Completed sample: 3000
Completed sample: 4000
Completed sample: 5000
Completed sample: 6000


In [7]:
del hg38
import gc
gc.collect()

0

Create Trinucleotide Spectra Figures
===



In [9]:
from siglib.spectrum import spectrum_map
from siglib.figures import py_labels
import matplotlib as mpl
import matplotlib.pyplot as plt

cosmic_search_url = "http://cancer.sanger.ac.uk/cosmic/search?q={}"
data = []

# Sort based on similarity attribute
sorted_results = sorted(list(merged_samples.values()),
                        reverse=True,
                        key=lambda x: x.similarity)

# Count to 100 and append to data
for i, x in enumerate(sorted_results):
    if i >= 100: break;
    if i % 5 == 0: print('Completed Figure {}'.format(i))

    data.append([i + 1,
                 '<a href={} target="_blank">{}</a>'.format(cosmic_search_url.format(x.name), x.name),
                 '{:0.2f}'.format(float(x.similarity)), 
                 x.total_mutations, 
                 x.histology['primary'].capitalize(),
                 x.site['primary'].capitalize()])
    if i < 100:
        continue
    spectrum_map(x=2, y=1,
                 x_inches=20,
                 dpi=420,
                 heights=[x.spectrum.values(), C10],
                 xlabels=[[y.parts[1] for y in x.spectrum.keys()],
                          [y.parts[1] for y in x.spectrum.keys()]],
                 labels=py_labels,
                 titles=[x.name + '\n', 'C10\n'])
    plt.savefig('/home/clint/siglib/figures/{}.png?raw=true'.format(x.name), bbox_inches='tight', dpi=150)
    mpl.pyplot.close("all")
    

Completed Figure 0
Completed Figure 5
Completed Figure 10
Completed Figure 15
Completed Figure 20
Completed Figure 25
Completed Figure 30
Completed Figure 35
Completed Figure 40
Completed Figure 45
Completed Figure 50
Completed Figure 55
Completed Figure 60
Completed Figure 65
Completed Figure 70
Completed Figure 75
Completed Figure 80
Completed Figure 85
Completed Figure 90
Completed Figure 95


In [10]:
from collections import Counter
from IPython.display import HTML, display

site_counter = Counter()

# Count all site occurences in merged_samples
for x in merged_samples.values():
    if x.site['primary'] in ['', 'NS']:
        continue
    site_counter.update([x.site['primary']])

display(HTML('<p><h1>{} Tumour Samples by Primary Site with > {} Mutations</h1>'
             ''.format(len(merged_samples), cutoff)))
display(HTML(
'<table><tr>'
'</tr><td><b>Primary Site</b></td><td><b>Frequency</b></td>'
    '<tr>{}</tr></table>'.format('</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in site_counter.most_common())
    )
))

Primary Site,Frequency
large_intestine,1014
liver,869
lung,859
skin,552
oesophagus,494
breast,402
stomach,367
kidney,363
urinary_tract,258
pancreas,238


In [11]:
display(HTML('<p><h1>Top 100 Tumour Samples Most Similar to A10</h1></p>'.format(len(merged_samples), cutoff)))

render = ''
image_repo = 'https://github.com/essigmannlab/aflatoxin_biomarker/blob/master/supplemental/cosmic_comparison_figures/{}.png?raw=true'

header = '</td></b><td><b>'.join(['Rank', 'Name', 'Similarity (cosine)',
                                  'Number of Mutations', 'Site Primary',
                                  'Histology Primary'])

for row in data:
    render += '<tr><td><b>'
    render += header
    render += '</b></td></tr>'

    render += '<tr><td>'
    render += '</td><td>'.join(str(_) for _ in row)
    render += '</td></tr>'

    render += '<tr><td colspan="6">' 
    name = image_repo.format(row[1]).split('>')[1].split('<')[0]

    render += '<img src="{}" style="text-align: center;">'.format(image_repo.format(name))
    render += '</td></tr>'

display(HTML(
    '<table>{}</table>'.format(render)
))